**Parsing Notes**
1. The parsing states and transitions are not understood
2. Some entries seem to be missing context

In [157]:
from bs4 import BeautifulSoup as Soup
from IPython.display import display, HTML
import pandas as pd
import re
import pprint
import json

In [167]:
class Token:
    def __init__(self, name, value=''):
        self.name = name
        self.value = value
    
    def __repr__(self):
        return '{} : {}'.format(self.name, self.value)
        
# we will use these tokens:
# NOTICE -> when NOTICE IS HEREBY GIVEN is detected
# MATTER -> when IN THE MATTER OF is detected
def tokenize(fragment):
    soup = Soup(fragment)
    for para in soup.find_all('p'):
        text = para.get_text(strip=True)
        
        if 'notice is hereby given' in text.lower():
            yield Token('NOTICE')
            
            # 'in the matter' could be part of the same text
            # that contained the 'notice is hereby given'
            if 'in the matter' in text.lower():
                loc = text.lower().find('in the matter')
                yield Token('TEXT', text[:loc])
                yield Token('MATTER')
                yield Token('TEXT', ' ' + text[loc:])
            else:
                yield Token('TEXT', text) 
        elif 'in the matter' in text.lower():
            loc = text.lower().find('in the matter')
            yield Token('TEXT', text[:loc])
            yield Token('MATTER')
            yield Token('TEXT', ' ' + text[loc:])
        else:
            yield Token('TEXT', text)


In [168]:
fn = 'procPublicationRequest Oct-Dec 2014 (Updated) - Sheet1-2.csv'
rows = pd.read_csv(fn, header=0)

In [169]:
class State:
    def run(self):
        # implement this for printing trace statements
        assert 0, "implement me"
        
    def next(self, input):
        assert 0, "implement me"
        
    def flush(self, record):
        raise Exception('wtf!')


In [170]:
class Matters(State):
    
    def __init__(self):
        State.__init__(self)
        self.matter = ''
    
    def run(self):
        print('Process MATTERS')
        
    def flush(self, record):
        record['matters'].append({ 'matter' : self.matter })
        self.matter = ''
        
    def next(self, tok, record):
        if tok.name == 'TEXT':
            self.matter += tok.value
        if tok.name == 'MATTER':
            self.flush(record)
        return self
            

class Notice(State):
    
    def __init__(self, notice=''):
        State.__init__(self)
        self.notice = notice
        
    def run(self):
        print('NOTICE')
        
    def next(self, tok, record):
        if tok.name == 'TEXT':
            self.notice += tok.value
            return self
        if tok.name == 'MATTER':
            record['context'] = self.notice
            return Matters()
        return self
    
    def flush(self, record):
        record['error'] = 'no transition out of NOTICE'
        
    
class Init(State):
    
    def run(self):
        print('INIT')
        
    def next(self, tok, record):
        if tok.name == 'NOTICE':
            return Notice()
        return self
                                  
    def flush(self, record):
        record['error'] = 'no transition out of INIT'
                                  

In [171]:
agency = rows['AgencyName'] == "Citywide Administrative Services"
description = rows['TypeOfNoticeDescription'] == "Public Hearings"
target_rows = rows[agency & description]


In [172]:
def scrape(row):
    output = {}
    if not isinstance(row.AdditionalDescription, str):
        output = { 'error' : 'source is not a string: {}'.format(row.AdditionalDescription) }
    else:   
        state = Init()
        record = { 'matters' :[], 'context' : '' }
        for tok in tokenize(row.AdditionalDescription):
            state = state.next(tok, record)
        state.flush(record)
        output = record

    row['output'] = json.dumps(output)
    return row

In [173]:
processed_rows = target_rows.apply(scrape,1)

In [174]:
errors = []
for rec in processed_rows[['RequestID', 'output', 'AdditionalDescription']].values:
    id, output, desc = rec
    output = json.loads(output)
    if output.get('error', None):
        errors.append({'RequestID' : id, 'error': output, 'desc' : desc})
        continue
    print('RequestID: {}'.format(id))
    pprint.pprint(output)
    display(HTML(desc))
    
    display(HTML('<hr/>'))

RequestID: 20141006102
{'context': 'NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
            'DISPOSITIONS PUBLIC HEARING, in accordance with Section 824 of '
            'the New York City Charter, will be held on Wednesday October '
            '22, 2014 at 10:00 a.m., 22 Reade Street, 2ndfloor conference '
            'room, Borough of Manhattan',
 'matters': [{'matter': ' IN THE MATTER OFa lease for The City of New '
                        'York, as Tenant, of approximately 10,826 rentable '
                        'square feet of space on part of the second\xa0 '
                        'floor in a building located at 1 Teleport '
                        'Drive\xa0 (Block 2165, Lot 170), in the Borough '
                        'of Staten Island, for the Traffic Enforcement '
                        'Division of the Police Department to use as an '
                        'office.The proposed lease shall be for a period '
                        'of twenty (20) 

RequestID: 20141105101
{'context': 'NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
            'DISPOSITIONS PUBLIC HEARING, in accordance with Section 824 of '
            'the New York City Charter, will be held on November 24, 2014 at '
            '10:00 a.m., 22 Reade Street, 2ndfloor conference room, Borough '
            'of Manhattan.',
 'matters': [{'matter': ' IN THE MATTER OFa lease renewal and amendment '
                        'agreement for The City of New York, as Tenant, of '
                        'approximately 21,741 rentable square feet of '
                        'space on the tenth floor in a building located at '
                        '200 Varick Street (Block 520, Lot 1) in the '
                        'Borough of Manhattan for the Board of Elections '
                        'to use as an office, or any other use that the '
                        'Department of Citywide Administrative Services '
                        'may determine.The 

RequestID: 20141118109
{'context': 'NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
            'DISPOSITIONS PUBLIC HEARING, in accordance with Section 824 of '
            'the New York City Charter, will be held on December 10, 2014 at '
            '10:30 a.m., 22 Reade Street, 2ndfloor conference room, Borough '
            'of Manhattan, ',
 'matters': [{'matter': ' in the matter of a lease for the City of New '
                        'York, as tenant, of approximately 16,334 rentable '
                        'square feet of space on the entire fifth (5th) '
                        'floor of the building located at 118-35 Queens '
                        'Boulevard (Block 2270, Lot 41) in the Borough of '
                        'Queens for a Computerized Testing and '
                        'Applications Center, or for such other use as the '
                        'Commissioner of the Department of Citywide '
                        'Administrative Services m

In [166]:
err_summary = '<h1>{} Errors parsing Citywide Administrative Services::Public Hearings'.format(len(errors))
display(HTML(err_summary))
for error in errors:
    pprint.pprint(error)
    print('\n\n')
    desc = error['desc']
    if isinstance(desc, str):
        display(HTML(error['desc']))
    display(HTML('<hr/>'))


{'RequestID': 20140930111,
 'desc': '<p align=center"> &nbsp;</p> <p> <strong>NOTICE IS HEREBY GIVEN '
         'THAT A REAL PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC '
         'HEARING</strong>',
 'error': {'context': '',
           'error': 'no transition out of NOTICE',
           'matters': []}}





{'RequestID': 20141007108,
 'desc': '<p align=center"> <strong>Notice of Public Hearing</strong></p> '
         '<p> &nbsp;</p> <p> <strong>NOTICE IS HEREBY GIVEN THAT A REAL '
         'PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC HEARING</strong>',
 'error': {'context': '',
           'error': 'no transition out of NOTICE',
           'matters': []}}





{'RequestID': 20141030107,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}





{'RequestID': 20141031112,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}





{'RequestID': 20141103110,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}





...

{'RequestID': 20141201109,
 'desc': '<p align=center"> <strong>Notice of Public Hearing</strong></p> '
         '<p> &nbsp;</p> <p> <strong>NOTICE IS HEREBY GIVEN THAT A REAL '
         'PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC HEARING</strong>',
 'error': {'context': '',
           'error': 'no transition out of NOTICE',
           'matters': []}}





{'RequestID': 20141203109,
 'desc': nan,
 'error': {'error': 'source is not a string: nan'}}





{'RequestID': 20141219102,
 'desc': '<p align=center"> <strong>Notice of Public Hearing</strong></p> '
         '<p> &nbsp;</p> <p> <strong>NOTICE IS HEREBY GIVEN THAT A REAL '
         'PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC HEARING</strong>',
 'error': {'context': '',
           'error': 'no transition out of NOTICE',
           'matters': []}}





{'RequestID': 20141219106,
 'desc': '<p align=center"> &nbsp;</p> <p> <strong>NOTICE IS HEREBY GIVEN '
         'THAT A REAL PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC '
         'HEARING</strong>',
 'error': {'context': '',
           'error': 'no transition out of NOTICE',
           'matters': []}}





{'RequestID': 20141223103,
 'desc': '<p align=center"> <strong>Corrected Notice of Public '
         'Hearing</strong></p> <p> &nbsp;</p> <p> <strong>NOTICE IS HEREBY '
         'GIVEN THAT A REAL PROPERTY ACQUISITIONS AND DISPOSITIONS PUBLIC '
         'HEARING</strong>',
 'error': {'context': '',
           'error': 'no transition out of NOTICE',
           'matters': []}}



